#### 进程和线程

python执行多任务，就会3个方法：
- 多进程
- 多线程
- 多进程 + 多线程

线程是最小的执行单元，而进程由至少一个线程组成。如何调度进程和线程，是由操作系统决定，程序不能决定什么时候助兴和执行多长时间。
多进程和多线程的程序涉及到同步、数据共享的问题，编写起来比较复杂

In [1]:
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (18352) start...


AttributeError: module 'os' has no attribute 'fork'

#### multiprocessing

In [3]:
from multiprocessing import Process

def run_func(name):
    print('Run process will %s (%s)...' % (name, os.getpid()))

if __name__ == "__main__":
    print("Parent process is %s" % (os.getpid()))
    p = Process(target = run_func, args=("test", ))
    print("process start")
    p.start()
    p.join()
    print("process end")

Parent process is 18352
process start
process end


In [4]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

Parent process 18352.
Child process will start.
Child process end.


#### Pool

In [6]:
from multiprocessing import Pool
import os, time, random

def long_time_tast(name):
    print("%s child task is %s" % (name, os.getpid()))
    start_time = time.time()
    time.sleep(random.random() * 3)
    end_time = time.time()
    print("The task has process for %d" % (end_time - start_time))
    
if __name__=="__main__":
    print("Parent is %s" % (os.getpid()))
    with Pool(4) as p:
        p.apply_async(long_time_tast, args = ("test", ))
    p.close()
    p.join()
    print("Process end")
    

Parent is 18352
Process end


#### 子进程

In [7]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [8]:
import subprocess

print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

$ nslookup


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 2: invalid continuation byte

#### 进程间通信

In [15]:
from multiprocessing import Process, Queue
import os, time, random

def write(q):
    print("The process is %s"%(os.getpid()))
    for element in ['A', 'B', 'C']:
        print("Put %s to queue ..." % value)
        q.put(element)
        time.sleep(random.random())

def read(q):
    print("The process is %s"%(os.getpid()))
    while True:
        value = q.get(True)
        print(value)

if __name__=="__main__":
    q = Queue()
    pw = Process(target=write, args = (q, ))
    pr = Process(target=read, args = (q, ))
    pw.start()
    pr.start()
    pw.join()
    pr.terminate()



In [14]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

#### 小结：

在Unix/Linux下，可以使用fork()调用实现多进程
要实现跨平台的多进程，可以使用multiprocessing模块
进程间通讯是通过Queue和Pipes等实现